In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import datetime
from dateutil.parser import parse

In [2]:
all_files_s3p = glob.glob(os.path.join("s3p" , "*/*.json"))
s3p_df_from_each_file = (pd.read_json(f) for f in all_files_s3p)
s3p_df = pd.concat(s3p_df_from_each_file, ignore_index=True)

all_files_s3s = glob.glob(os.path.join("s3s" , "*/*.json"))
s3s_df_from_each_file = (pd.read_json(f) for f in all_files_s3s)
s3s_df = pd.concat(s3s_df_from_each_file, ignore_index=True)

# Procesar particulares sancionados

In [3]:
def parse_date(x):
    try:
        return parse(x)
    except:
        return np.datetime64('NaT')

In [4]:
sancionados = pd.json_normalize(s3p_df.particularSancionado)
inhabilitacion = pd.json_normalize(s3p_df.inhabilitacion)
inhabilitacion.fechaInicial =   inhabilitacion.fechaInicial.apply(parse_date)
inhabilitacion.fechaFinal = inhabilitacion.fechaFinal.apply(parse_date)

In [5]:
s3p_df["sancion_nombre"] = sancionados.nombreRazonSocial
s3p_df["sancion_tipoPersona"] = sancionados.tipoPersona
s3p_df["sancion_objetoSocial"] = sancionados.objetoSocial
s3p_df["inhabilitacion_fechaInicial"] = inhabilitacion.fechaInicial
s3p_df["inhabilitacion_fechaFinal"] = inhabilitacion.fechaFinal
s3p_df["sancion_nombre"] = s3p_df["sancion_nombre"].str.lower()
s3p_df["sancion_nombre"] = s3p_df["sancion_nombre"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [6]:
columnas = ["id", "expediente", "sancion_nombre", "sancion_tipoPersona", "sancion_objetoSocial", "inhabilitacion_fechaInicial","inhabilitacion_fechaFinal"]
s3p_df = s3p_df[columnas]
s3p_df["tipo_persona"] = "particular"
s3p_df[6:10]

,id,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona
6,652dbfd33571bce3cc3d002c,32/2020,mishael cayetano pina,F,PERSONA FÍSICA,NaT,NaT,particular
7,29,SE/18/2019/JQII/AR,alejandro valerio payan,F,-,2020-10-23,2024-10-22,particular
8,28,SE/18/2019/JQII/AR,"horus grupo empresarial, sociedad anonima de c...",M,LA CONSTRUCCION Y DISEÑO DE TODA CLASE DE OBRA...,2020-10-23,2025-12-07,particular
9,61e1db512a16a00013957b07,TJA/QR-SU3-092-2019,juan carlos ibarra guevara,F,NaN,NaT,NaT,particular


# Procesar servidores sancionados

In [7]:
servidorPublicoSancionado = pd.json_normalize(s3s_df.servidorPublicoSancionado)
inhabilitacion_servidor = pd.json_normalize(s3s_df.inhabilitacion)

In [8]:
s3s_df.columns

Index(['id', 'fechaCaptura', 'expediente', 'institucionDependencia',
       'servidorPublicoSancionado', 'autoridadSancionadora', 'tipoFalta',
       'tipoSancion', 'causaMotivoHechos', 'resolucion', 'multa',
       'inhabilitacion', 'documentos', 'observaciones', '__v'],
      dtype='object')

In [9]:
servidorPublicoSancionado[["nombres", "primerApellido", "segundoApellido" ]] = servidorPublicoSancionado[["nombres", "primerApellido", "segundoApellido" ]].fillna("")
servidorPublicoSancionado["nombre"] = servidorPublicoSancionado.nombres + " " + servidorPublicoSancionado.primerApellido + " " + servidorPublicoSancionado.segundoApellido
servidorPublicoSancionado["nombre"] = servidorPublicoSancionado["nombre"].str.lower().str.strip()
servidorPublicoSancionado["nombre"] = servidorPublicoSancionado["nombre"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [10]:
servidorPublicoSancionado.isna().sum()

nombres               0
primerApellido        0
segundoApellido       0
puesto                0
nivel              3199
genero.clave       5159
genero.valor       5159
curp               5547
rfc                5547
nombre                0
dtype: int64

In [11]:
inhabilitacion_servidor.fechaInicial =  inhabilitacion_servidor.fechaInicial.apply(parse_date)
inhabilitacion_servidor.fechaFinal = inhabilitacion_servidor.fechaFinal.apply(parse_date)

In [12]:
inhabilitacion_servidor.isna().sum()

plazo           864
fechaInicial    873
fechaFinal      884
dtype: int64

In [13]:
mask = (~inhabilitacion_servidor.fechaInicial.isna()) & (inhabilitacion_servidor.fechaFinal.isna())  
inhabilitacion_servidor[mask]

,plazo,fechaInicial,fechaFinal
3276,Dato no proporcionado,2014-08-14 00:00:00,NaT
3340,Dato no proporcionado,2014-08-15 00:00:00,NaT
3381,Dato no proporcionado,2014-08-15 00:00:00,NaT
4118,Dato no proporcionado,2014-08-15 00:00:00,NaT
4160,Dato no proporcionado,2014-08-15 00:00:00,NaT
4239,Dato no proporcionado,2014-08-15 00:00:00,NaT
4456,Dato no proporcionado,1998-11-21 00:00:00,NaT
4678,Dato no proporcionado,2014-08-13 00:00:00,NaT
5063,Dato no proporcionado,2014-08-15 00:00:00,NaT
5122,Dato no proporcionado,2001-08-08 00:00:00,NaT


In [14]:
s3s_df["sancion_nombre"] = servidorPublicoSancionado["nombre"]
s3s_df["inhabilitacion_fechaInicial"] = inhabilitacion_servidor.fechaInicial
s3s_df["inhabilitacion_fechaFinal"] = inhabilitacion_servidor.fechaFinal

In [15]:
columnas = ["id", "expediente", "sancion_nombre", "inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal"]
s3s_df = s3s_df[columnas]
s3s_df["tipo_persona"] = "servidor_publico"
s3s_df.head()

,id,expediente,sancion_nombre,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona
0,b411f02b-5ea7-489e-93e5-9eca24d090f6,CA/012/2022,andres reyes enriquez,2022-02-15 00:00:00,2022-05-15 00:00:00,servidor_publico
1,65b3929d-b116-4ba8-9eb6-398baca819fb,CA/001/2022,karina lizbeth valencia perez,2021-12-30 00:00:00,2022-03-30 00:00:00,servidor_publico
2,b6766222-8804-47f2-b008-f9d693f9133c,CA/007/2022,venamar ramos vera,2022-02-01 00:00:00,2022-05-01 00:00:00,servidor_publico
3,67bf0c34-1899-4185-9aba-7a1c03bd142b,CA/011/2022,brenda guadalupe becerra bello,2022-02-15 00:00:00,2022-05-15 00:00:00,servidor_publico
4,1982c64d-e4bc-4ec5-b1bf-cd80c7bc3689,CA/019/2021,armando salazar gonzalez,2021-08-02 00:00:00,2022-02-02 00:00:00,servidor_publico


# Combinar datasets

In [16]:
df = pd.concat([s3p_df,s3s_df]).reset_index(drop = True)
df.head()

,id,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona
0,a56b34a6-57c7-42b9-bb60-e214a65db391,PRA/68/2021,ofimagen sociedad anonima de capital variable,M,NaN,NaT,NaT,particular
1,622799f073ed865c36ba7cd5,07/2018,pedro barrueta escobar,F,FISICA,NaT,NaT,particular
2,622799f173ed865c36ba7cda,13/2019,monica virginia martinez bringas,F,PERSONA FÍSICA,NaT,NaT,particular
3,622799f173ed865c36ba7cdf,37/2020,rodriguez venegas iris ivonne,F,SD,NaT,NaT,particular
4,6317b5dd055a2a7017d97424,01/2021,"proyectos y construcciones r&m, sociedad anon...",F,"LA CONSTRUCCIÓN, REMODELACIÓN, MANTENIMIENTO, ...",NaT,NaT,particular


In [17]:
df.to_pickle("inhabilitaciones.pkl")